<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Elegibility-Criteria/blob/main/Roberta%2BLLM/compare_two_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# uncomment if using colab
!pip install -q -U datasets
!pip install seqeval
!pip install -q -U evaluate
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U bitsandbytes
# !pip install -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=3d05e378a95e6360b53d3fd878ed43d9796294678465f3f139c5e30bef6ab718
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextGenerationPipeline
import torch
import accelerate
import os
from utils import *

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict

In [3]:
from datasets import load_dataset, load_metric

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# dict for the entities (entity to int value)
simple_ent = {"Condition", "Value", "Drug", "Procedure", "Measurement", "Temporal", "Observation", "Person", "Device"}
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Drug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16,
    "B-Device": 17,
    "I-Device": 18
}

entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [6]:
root = '..'
root = './drive/MyDrive/TER-LISN-2024'
data_path = f'{root}/data'
models_path = f'{root}/models'

In [7]:
model_name = "mistralai/Mistral-7B-v0.1"

In [8]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# import tokenizer for mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [10]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [11]:
dataset = load_dataset('JavierLopetegui/chia_v1')

In [12]:
# for each sentence save the text
def generate_sentences_from_tokens(sentences):
    texts_sentences = []
    sentences_tokens = sentences['tokens']
    for sentence in sentences_tokens:
        sent_text = " ".join(sentence)
        texts_sentences.append(sent_text)
    sentences['text'] = texts_sentences
    return sentences

In [13]:
def build_prompts(sentences, prompt_type=2):
    sentences_prompts = []
    for sent in sentences['text']:
        prompt = build_prompt(sent, prompt_type)
        sentences_prompts.append(prompt)
    sentences['prompt'] = sentences_prompts
    return sentences

In [14]:
dataset = dataset.map(lambda x: generate_sentences_from_tokens(x), batched = True)
dataset_prompt1 = dataset.map(lambda x: build_prompts(x, prompt_type=1), batched = True)
dataset_prompt2 = dataset.map(lambda x: build_prompts(x, prompt_type=2), batched = True)

In [15]:
test_dataset_p1 = dataset_prompt1['test']
test_dataset_p2 = dataset_prompt2['test']

In [16]:
test_dataset_p1['prompt'][0]

'I need to perform a named entity recognition task on a  text related with inclusion criteria in clinical trials.\n    The entities you need to recognize are: Condition, Value, Drug, Procedure, Measurement, Temporal, Observation, Person, Mood, Device and Pregnancy_considerations.\n    Particularly you have to produce the ouput in the BIO format. I will show you an example of the expected output.\n    Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement\n    Output:\n    Patients O\n    with O\n    symptomatic O\n    CNS B-Condition\n    metastases I-Condition\n    or O\n    leptomeningeal B-Condition\n    involvement I-Condition\n\n    You can see that tokens without any entity are labeled as O, and the tokens that are part of an entity are labeled as B-<entity> or I-<entity> depending on if they are the beginning or the inside of the entity.\n    Please, just answer the question for this specific example and stop writting after that.\n    Input text: self

In [17]:
test_dataset_p2['prompt'][0]

'I am working on a named entity recognition problem, in the context of clinical\n    trials eligibility criteria. I will show you the list of entities:\n    - Condition\n    - Value\n    - Drug\n    - Procedure\n    - Measurement\n    - Temporal\n    - Observation\n    - Person\n    - Mood\n    - Device\n\n    Your task consists in annotate the named entities in a given sentence in the format I will explain you.\n    I will explain you with some examples:\n\n    Example 1:\n    Input: Patients who have received prior chemotherapy for unresectable disease.\n    Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.\n\n    Example 2:\n    Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.\n    Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the inve

In [ ]:
# # keep just the prompt column
# test_dataset_p1 = test_dataset_p1.remove_columns(['tokens', 'text', 'ner_tags', 'file'])
# test_dataset_p2 = test_dataset_p2.remove_columns(['tokens', 'text', 'ner_tags', 'file'])

In [ ]:
# data_loader_p1 = torch.utils.data.DataLoader(test_dataset_p1, batch_size=4, shuffle=False)
# data_loader_p2 = torch.utils.data.DataLoader(test_dataset_p2, batch_size=4, shuffle=False)

In [ ]:
# generated_sentences_p1 = []
# for batch in data_loader_p1:
#     generated_sentences_p1.extend(pipe(batch['prompt'], max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole"))

In [ ]:
# generated_sentences_p2 = []
# for batch in data_loader_p2:
#     generated_sentences_p2.extend(pipe(batch['prompt'], max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole"))

In [21]:
from tqdm import tqdm

In [31]:
# generation one by one
generated_sentences_p1 = []
for sentence in tqdm(test_dataset_p1['prompt'][:50]):
    output = pipe(sentence, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n\n')[0]
    generated_sentences_p1.append(output)

100%|██████████| 50/50 [33:08<00:00, 39.77s/it]


In [26]:
from tqdm import tqdm

In [28]:
generated_sentences_p2 = []
for sentence in tqdm(test_dataset_p2['prompt'][:50]):
    sentence += '\noutput: '
    output = pipe(sentence, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n')[0]
    generated_sentences_p2.append(output)

100%|██████████| 50/50 [34:05<00:00, 40.91s/it]


In [29]:
generated_sentences_p2[0]

' self - reported healthy adults between the ages of <Measurement>18 - 60</Measurement> who are fluent in <Language>English</Language>.'

In [30]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [31]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

**Standarizing true annotations**

In [32]:
import re

In [33]:
new_true_annotations = []
for sent in dataset['test']:
    annotation = []
    for word, tag in zip(sent['tokens'], sent['ner_tags']):
        annotation.append((word, entities_list[tag]))
    new_annotation = []
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))
        else:
            new_annotation.append((word, tag))
    new_true_annotations.append(new_annotation)
len(new_true_annotations)

1307

In [34]:
true_annotations = []
for sent in new_true_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    true_annotations.append(dicc_sent)
len(true_annotations)

1307

In [35]:
true_df = pd.DataFrame(true_annotations)
true_ann_dataset = Dataset.from_pandas(true_df)

In [36]:
true_ann_dataset = true_ann_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [37]:
true_ann_dataset['ner_tags'][0]

[0, 0, 0, 1, 15, 3, 4, 4, 4, 4, 4, 4, 0, 0, 13, 14, 14, 0]

**Evaluating prompt 1**

In [121]:
new_p1_annotations = []
for sent in generated_sentences_p1:
    annotation = []
    for line in sent.split('\n')[2:]:
        if line != '':
            splited_line = line.split()
            if len(splited_line) > 2:
                splited_line = [' '.join(splited_line[:-1]), splited_line[-1]]
            if len(splited_line) != 2:
                continue
            word, tag = splited_line
            annotation.append((word, tag))
    new_annotation = []
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    for i,(word, tag) in enumerate(annotation):
        if tag not in sel_ent:
            tag = "O"
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    if not tag.startswith('I') and not tag.startswith('B'):
                        label = "O"
                    else:
                        label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))
        else:
            new_annotation.append((word, tag))
    new_p1_annotations.append(new_annotation)
len(new_p1_annotations)

In [73]:
p1_annotations = []
for sent in new_p1_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    p1_annotations.append(dicc_sent)
len(p1_annotations)

50

In [74]:
p1_annotations

[{'tokens': ['self',
   '-',
   'reported',
   'healthy',
   'adults',
   'between',
   'the',
   'ages',
   'of',
   '18',
   '-',
   '60',
   'who',
   'are',
   'fluent',
   'in',
   'English',
   '.'],
  'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['Treatment',
   'with',
   'any',
   'investigational',
   'drug',
   'within',
   '30',
   'days',
   'of',
   'entry',
   'to',
   'this',
   'protocol'],
  'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['Current', 'treatment', 'with', 'Telbivudine'],
  'ner_tags': [0, 0, 0, 5]},
 {'tokens': ['Severe',
   'hepatitis',
   'activity',
   'as',
   'documented',
   'by',
   'ALT>10 x ULN'],
  'ner_tags': [0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['History',
   'of',
   'decompensated',
   'cirrhosis',
   '(',
   'defined',
   'as',
   'jaundice',
   'in',
   'the presence',
   'of',
   'cirrhosis',
   ',',
   'ascites',
   ',',
   'bleeding',
   'gastric',
   'or',
   'esophageal',
   'var

In [75]:
p1_df = pd.DataFrame(p1_annotations)
p1_dataset = Dataset.from_pandas(p1_df)

In [76]:
p1_dataset = p1_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [78]:
# keep just sentences with the same length
sentences_to_evaluate_p1 = []
sentences_to_evaluate_true = []

for i in range(len(p1_dataset)):
    if len(p1_dataset['labels'][i]) == len(true_ann_dataset['labels'][i]):
        sentences_to_evaluate_p1.append(p1_dataset['labels'][i])
        sentences_to_evaluate_true.append(true_ann_dataset['labels'][i])

In [79]:
print(len(sentences_to_evaluate_p1)/len(p1_dataset))

0.72


In [38]:
def get_labels(p):
    predictions, labels = p
    # Remove ignored index (special tokens)
    predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return predictions, labels

In [81]:
pred_labels, true_labels = get_labels((sentences_to_evaluate_p1, sentences_to_evaluate_true))

In [83]:
from eval_file import *

In [39]:
# from eval_file import *

import argparse
from collections import defaultdict
from itertools import chain
from math import pow
from pathlib import Path

# from common_utils.common_io import load_bio_file_into_sents
# from common_utils.common_log import create_logger
# -*- coding: utf-8 -*-

# -*- coding: utf-8 -*-

import json
import pickle as pkl


def read_from_file(ifn):
    with open(ifn, "r") as f:
        text = f.read()
    return text


def write_to_file(text, ofn):
    with open(ofn, "w") as f:
        f.write(text)
    return True


def pkl_load(ifn):
    with open(ifn, "rb") as f:
        pdata = pkl.load(f)
    return pdata


def pkl_dump(pdata, ofn):
    with open(ofn, "wb") as f:
        pkl.dump(pdata, f)
    return True


def json_load(ifn):
    with open(ifn, "r") as f:
        jdata = json.load(f)
    return jdata


def json_dump(jdata, ofn):
    with open(ofn, "w") as f:
        json.dump(jdata, f)
    return True


def load_bio_file_into_sents(bio_file, word_sep=" ", do_lower=False):
    bio_text = read_from_file(bio_file)
    bio_text = bio_text.strip()
    if do_lower:
        bio_text = bio_text.lower()

    new_sents = []
    sents = bio_text.split("\n\n")

    for sent in sents:
        new_sent = []
        words = sent.split("\n")
        for word in words:
            new_word = word.split(word_sep)
            new_sent.append(new_word)
        new_sents.append(new_sent)

    return new_sents


def output_bio(bio_data, output_file, sep=" "):
    with open(output_file, "w") as f:
        for sent in bio_data:
            for word in sent:
                line = sep.join(word)
                f.write(line)
                f.write("\n")
            f.write("\n")


class PRF:
    def __init__(self):
        self.true = 0
        self.false = 0

    def add_true_case(self):
        self.true += 1

    def add_false_case(self):
        self.false += 1

    def get_true_false_counts(self):
        return self.true, self.false

    def __str__(self):
        return str(self.__dict__)


class BioEval:
    def __init__(self):
        self.acc = PRF()
        # prediction
        self.all_strict = PRF()
        self.all_relax = PRF()
        self.cat_strict = defaultdict(PRF)
        self.cat_relax = defaultdict(PRF)
        # gold standard
        self.gs_all = 0
        self.gs_cat = defaultdict(int)
        self.performance = dict()
        self.counts = dict()
        self.beta = 1
        self.label_not_for_eval = {'o'}

    def reset(self):
        self.acc = PRF()
        self.all_strict = PRF()
        self.all_relax = PRF()
        self.cat_strict = defaultdict(PRF)
        self.cat_relax = defaultdict(PRF)
        self.gs_all = 0
        self.gs_cat = defaultdict(int)
        self.performance = dict()
        self.counts = dict()

    def set_beta_for_f_score(self, beta):
        print("Using beta={} for calculating F-score".format(beta))
        self.beta = beta

    # def set_logger(self, logger):
    #     self.logger = logger

    def add_labels_not_for_eval(self, *labels):
        for each in labels:
            self.label_not_for_eval.add(each.lower())

    def __calc_prf(self, tp, fp, tp_tn):
        """
        Using this function to calculate F-beta score, beta=1 is f_score-score, set beta=2 favor recall, and set beta=0.5 favor precision.
        Using set_beta_for_f_score function to change beta value.
        """
        tp_fp = tp + fp
        pre = 1.0 * tp / tp_fp if tp_fp > 0 else 0.0
        rec = 1.0 * tp / tp_tn if tp_tn > 0 else 0.0
        beta2 = pow(self.beta, 2)
        f_beta = (1 + beta2) * pre * rec / (beta2 * pre + rec) if (pre + rec) > 0 else 0.0
        return pre, rec, f_beta

    def __measure_performance(self):
        self.performance['overall'] = dict()

        acc_true_num, acc_false_num = self.acc.get_true_false_counts()
        total_acc_num = acc_true_num + acc_false_num
        # calc acc
        overall_acc = round(1.0 * acc_true_num / total_acc_num, 4) if total_acc_num > 0 else 0.0
        self.performance['overall']['acc'] = overall_acc

        strict_true_counts, strict_false_counts = self.all_strict.get_true_false_counts()
        strict_pre, strict_rec, strict_f_score = self.__calc_prf(strict_true_counts, strict_false_counts, self.gs_all)
        self.performance['overall']['strict'] = dict()
        self.performance['overall']['strict']['precision'] = strict_pre
        self.performance['overall']['strict']['recall'] = strict_rec
        self.performance['overall']['strict']['f_score'] = strict_f_score

        relax_true_counts, relax_false_counts = self.all_relax.get_true_false_counts()
        relax_pre, relax_rec, relax_f_score = self.__calc_prf(relax_true_counts, relax_false_counts, self.gs_all)
        self.performance['overall']['relax'] = dict()
        self.performance['overall']['relax']['precision'] = relax_pre
        self.performance['overall']['relax']['recall'] = relax_rec
        self.performance['overall']['relax']['f_score'] = relax_f_score

        self.performance['category'] = dict()
        self.performance['category']['strict'] = dict()
        for k, v in self.cat_strict.items():
            self.performance['category']['strict'][k] = dict()
            stc, sfc = v.get_true_false_counts()
            p, r, f = self.__calc_prf(stc, sfc, self.gs_cat[k])
            self.performance['category']['strict'][k]['precision'] = p
            self.performance['category']['strict'][k]['recall'] = r
            self.performance['category']['strict'][k]['f_score'] = f

        self.performance['category']['relax'] = dict()
        for k, v in self.cat_relax.items():
            self.performance['category']['relax'][k] = dict()
            rtc, rfc = v.get_true_false_counts()
            p, r, f = self.__calc_prf(rtc, rfc, self.gs_cat[k])
            self.performance['category']['relax'][k]['precision'] = p
            self.performance['category']['relax'][k]['recall'] = r
            self.performance['category']['relax'][k]['f_score'] = f

    def __measure_counts(self):
        # gold standard
        self.counts['expect'] = dict()
        self.counts['expect']['overall'] = self.gs_all
        for k, v in self.gs_cat.items():
            self.counts['expect'][k] = v
        # prediction
        self.counts['prediction'] = {'strict': dict(), 'relax': dict()}
        # strict
        strict_true_counts, strict_false_counts = self.all_strict.get_true_false_counts()
        self.counts['prediction']['strict']['overall'] = dict()
        self.counts['prediction']['strict']['overall']['total'] = strict_true_counts + strict_false_counts
        self.counts['prediction']['strict']['overall']['true'] = strict_true_counts
        self.counts['prediction']['strict']['overall']['false'] = strict_false_counts
        for k, v in self.cat_strict.items():
            t, f = v.get_true_false_counts()
            self.counts['prediction']['strict'][k] = dict()
            self.counts['prediction']['strict'][k]['total'] = t + f
            self.counts['prediction']['strict'][k]['true'] = t
            self.counts['prediction']['strict'][k]['false'] = f
        # relax
        relax_true_counts, relax_false_counts = self.all_relax.get_true_false_counts()
        self.counts['prediction']['relax']['overall'] = dict()
        self.counts['prediction']['relax']['overall']['total'] = relax_true_counts + relax_false_counts
        self.counts['prediction']['relax']['overall']['true'] = relax_true_counts
        self.counts['prediction']['relax']['overall']['false'] = relax_false_counts
        for k, v in self.cat_relax.items():
            t, f = v.get_true_false_counts()
            self.counts['prediction']['relax'][k] = dict()
            self.counts['prediction']['relax'][k]['total'] = t + f
            self.counts['prediction']['relax'][k]['true'] = t
            self.counts['prediction']['relax'][k]['false'] = f

    @staticmethod
    def __strict_match(gs, pred, s_idx, e_idx, en_type):
        if e_idx < len(gs) and gs[e_idx] == f"i-{en_type}":
            # check token after end in GS is not continued entity token
            return False
        elif gs[s_idx] != f"b-{en_type}" or pred[s_idx] != f"b-{en_type}":
            # force first token to be B-
            return False
        # check every token in span is the same
        for idx in range(s_idx, e_idx):
            if gs[idx] != pred[idx]:
                return False
        return True

    @staticmethod
    def __relax_match(gs, pred, s_idx, e_idx, en_type):
        # we adopt the partial match strategy which is very loose compare to right-left or approximate match
        for idx in range(s_idx, e_idx):
            gs_cate = gs[idx].split("-")[-1]
            pred_bound, pred_cate = pred[idx].split("-")
            if gs_cate == pred_cate == en_type:
                return True
        return False

    @staticmethod
    def __check_evaluated_already(gs_dict, cate, start_idx, end_idx):
        for k, v in gs_dict.items():
            c, s, e = k
            if not (e < start_idx or s > end_idx) and c == cate:
                if v == 0:
                    return True
                else:
                    gs_dict[k] -= 1
                    return False
        return False

    def __process_bio(self, gs_bio, pred_bio):
        # measure acc
        for w_idx, (gs_word, pred_word) in enumerate(zip(gs_bio, pred_bio)):
            # measure acc
            if gs_word == pred_word:
                self.acc.add_true_case()
            else:
                self.acc.add_false_case()

        # process gold standard
        llen = len(gs_bio)
        gs_dict = defaultdict(int)
        cur_idx = 0
        while cur_idx < llen:
            if gs_bio[cur_idx].strip() in self.label_not_for_eval:
                cur_idx += 1
            else:
                start_idx = cur_idx
                end_idx = start_idx + 1
                _, cate = gs_bio[start_idx].strip().split('-')
                while end_idx < llen and gs_bio[end_idx].strip() == f"i-{cate}":
                    end_idx += 1
                self.gs_all += 1
                self.gs_cat[cate] += 1
                gs_dict[(cate, start_idx, end_idx)] += 1
                cur_idx = end_idx
        # process predictions
        cur_idx = 0
        while cur_idx < llen:
            if pred_bio[cur_idx].strip() in self.label_not_for_eval:
                cur_idx += 1
            else:
                start_idx = cur_idx
                end_idx = start_idx + 1
                _, cate = pred_bio[start_idx].strip().split("-")
                while end_idx < llen and pred_bio[end_idx].strip() == f"i-{cate}":
                    end_idx += 1
                if self.__strict_match(gs_bio, pred_bio, start_idx, end_idx, cate):
                    self.all_strict.add_true_case()
                    self.cat_strict[cate].add_true_case()
                    self.all_relax.add_true_case()
                    self.cat_relax[cate].add_true_case()
                elif self.__relax_match(gs_bio, pred_bio, start_idx, end_idx, cate):
                    if self.__check_evaluated_already(gs_dict, cate, start_idx, end_idx):
                        cur_idx = end_idx
                        continue
                    self.all_strict.add_false_case()
                    self.cat_strict[cate].add_false_case()
                    self.all_relax.add_true_case()
                    self.cat_relax[cate].add_true_case()
                else:
                    self.all_strict.add_false_case()
                    self.cat_strict[cate].add_false_case()
                    self.all_relax.add_false_case()
                    self.cat_relax[cate].add_false_case()
                cur_idx = end_idx

    def eval_file(self, gs_file, pred_file):
        print("processing gold standard file: {} and prediciton file: {}".format(gs_file, pred_file))
        pred_bio_sents = load_bio_file_into_sents(pred_file, do_lower=True)
        gs_bio_sents = load_bio_file_into_sents(gs_file, do_lower=True)
        # process bio data
        # check two data have same amount of sents
        assert len(gs_bio_sents) == len(pred_bio_sents), \
            "gold standard and prediction have different dimension: gs: {}; pred: {}".format(len(gs_bio_sents), len(pred_bio_sents))
        # measure performance
        for s_idx, (gs_sent, pred_sent) in enumerate(zip(gs_bio_sents, pred_bio_sents)):
            # check two sents have same No. of words
            assert len(gs_sent) == len(pred_sent), \
                "In {}th sentence, the words counts are different; gs: {}; pred: {}".format(s_idx, gs_sent, pred_sent)
            gs_sent = list(map(lambda x: x[-1], gs_sent))
            pred_sent = list(map(lambda x: x[-1], pred_sent))
            self.__process_bio(gs_sent, pred_sent)
        # get the evaluation matrix
        self.__measure_performance()
        self.__measure_counts()

    def eval_mem(self, gs, pred, do_flat=False):
        # flat sents to sent; we assume input sequences only have 1 dimension (only labels)
        if do_flat:
            print('Sentences have been flatten to 1 dim.')
            gs = list(chain(*gs))
            pred = list(chain(*pred))
            gs = list(map(lambda x: x.lower(), gs))
            pred = list(map(lambda x: x.lower(), pred))
            self.__process_bio(gs, pred)
        else:
            for sidx, (gs_s, pred_s) in enumerate(zip(gs, pred)):
                gs_s = list(map(lambda x: x.lower(), gs_s))
                pred_s = list(map(lambda x: x.lower(), pred_s))
                self.__process_bio(gs_s, pred_s)

        self.__measure_performance()
        self.__measure_counts()

    def evaluate_annotations(self, gs, pred, do_lower=False):
        for gs_sent, pred_sent in zip(gs, pred):
            if do_lower:
              gs_sent = list(map(lambda x: x.lower(), gs_sent))
              pred_sent = list(map(lambda x: x.lower(), pred_sent))
            self.__process_bio(gs_sent, pred_sent)

        self.__measure_performance()
        self.__measure_counts()

    def get_performance(self):
        return self.performance

    def get_counts(self):
        return self.counts

    def save_evaluation(self, file):
        with open(file, "w") as f:
            json.dump(self.performance, f)

    def show_evaluation(self, digits=4):
        if len(self.performance) == 0:
            raise RuntimeError('call eval_mem() first to get the performance attribute')

        cate = self.performance['category']['strict'].keys()

        headers = ['precision', 'recall', 'f1']
        width = max(max([len(c) for c in cate]), len('overall'), digits)
        head_fmt = '{:>{width}s} ' + ' {:>9}' * len(headers)

        report = head_fmt.format(u'', *headers, width=width)
        report += '\n\nstrict\n'

        row_fmt = '{:>{width}s} ' + ' {:>9.{digits}f}' * 3 + '\n'
        for c in cate:
            precision = self.performance['category']['strict'][c]['precision']
            recall = self.performance['category']['strict'][c]['recall']
            f1 = self.performance['category']['strict'][c]['f_score']
            report += row_fmt.format(c, *[precision, recall, f1], width=width, digits=digits)

        report += '\nrelax\n'

        for c in cate:
            precision = self.performance['category']['relax'][c]['precision']
            recall = self.performance['category']['relax'][c]['recall']
            f1 = self.performance['category']['relax'][c]['f_score']
            report += row_fmt.format(c, *[precision, recall, f1], width=width, digits=digits)

        report += '\n\noverall\n'
        report += 'acc: ' + str(self.performance['overall']['acc'])
        report += '\nstrict\n'
        report += row_fmt.format('', *[self.performance['overall']['strict']['precision'],
                                       self.performance['overall']['strict']['recall'],
                                       self.performance['overall']['strict']['f_score']], width=width, digits=digits)

        report += '\nrelax\n'
        report += row_fmt.format('', *[self.performance['overall']['relax']['precision'],
                                       self.performance['overall']['relax']['recall'],
                                       self.performance['overall']['relax']['f_score']], width=width, digits=digits)
        return report


In [97]:
evaluator = BioEval()

In [119]:
evaluator.evaluate_annotations(true_labels[:50], pred_labels, do_lower=True)

In [106]:
evaluator.performance

{'overall': {'acc': 0.5583,
  'strict': {'precision': 0.4392764857881137,
   'recall': 0.5862068965517241,
   'f_score': 0.5022156573116692},
  'relax': {'precision': 0.5736434108527132,
   'recall': 0.7655172413793103,
   'f_score': 0.6558345642540621}},
 'category': {'strict': {'condition': {'precision': 0.48424068767908307,
    'recall': 0.7041666666666667,
    'f_score': 0.5738539898132428},
   'drug': {'precision': 0.07142857142857142,
    'recall': 0.07692307692307693,
    'f_score': 0.07407407407407408},
   'value': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0},
   'temporal': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0},
   'person': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0},
   'measurement': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0}},
  'relax': {'condition': {'precision': 0.6160458452722063,
    'recall': 0.8958333333333334,
    'f_score': 0.7300509337860781},
   'drug': {'precision': 0.07142857142857142,
    'recall': 0.07692307692307693,
    'f_sco

In [120]:
evaluator.save_evaluation('eval_p1.json')

**Evaluating prompt 2**

In [60]:
def parse_ann2bio(sentence, pattern, pattern1, pattern2):
    # if sentence[-1] == "\n":
    #     sentence = sentence[:-2] # remove the \n and a final point wrongly added
    # else:
    #     sentence = sentence[:-1] # remove the final point wrongly added

    # find the entities
    occurrences = re.finditer(pattern, sentence)
    indexes = [(match.start(), match.end()) for match in occurrences]

    annotation = []
    i = 0


    # create the bio list
    for beg, end in indexes:
        if beg > i:
            annotation.extend([(word, "O") for word in sentence[i:beg].split()])
        entity = sentence[beg:end]
        entity_name = re.search(pattern1, entity).group(1)
        entity = entity.replace(f'<{entity_name}>', "").replace(f'</{entity_name}>', "")
        split_entity = entity.split()
        annotation.append((split_entity[0], "B-" + entity_name))
        annotation.extend([(word, "I-" + entity_name) for word in split_entity[1:]])
        i = end
    annotation.extend([(word, "O") for word in sentence[i:].split()])

    # check punctuation sign in tokens and put them as individual tokens
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    new_annotation = []
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))

        else:
            new_annotation.append((word, tag))


    return new_annotation

In [45]:
pattern1 = r'<(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern2 = r'</(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern = f'{pattern1}.*?{pattern2}'

In [59]:
generated_sentences_p2[2], dataset['test'][2]

(' Current treatment with <Drug>Telbivudine</Drug>',
 {'tokens': ['Current', 'treatment', 'with', 'Telbivudine'],
  'ner_tags': [0, 0, 0, 5],
  'file': 'NCT01373684_exc.bio.txt',
  'index': 1,
  'text': 'Current treatment with Telbivudine'})

In [61]:
new_p2_annotations = []
for sent in generated_sentences_p2:
    annotation = parse_ann2bio(sent, pattern, pattern1, pattern2)
    new_p2_annotations.append(annotation)
len(new_p2_annotations)

50

In [62]:
new_p2_annotations[0]

[('self', 'O'),
 ('-', 'O'),
 ('reported', 'O'),
 ('healthy', 'O'),
 ('adults', 'O'),
 ('between', 'O'),
 ('the', 'O'),
 ('ages', 'O'),
 ('of', 'O'),
 ('18', 'B-Measurement'),
 ('-', 'I-Measurement'),
 ('60', 'I-Measurement'),
 ('who', 'O'),
 ('are', 'O'),
 ('fluent', 'O'),
 ('in', 'O'),
 ('<Language>English</Language>', 'O'),
 ('.', 'O')]

In [63]:
p2_annotations = []
for sent in new_p2_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    p2_annotations.append(dicc_sent)
len(p2_annotations)

50

In [64]:
p2_df = pd.DataFrame(p2_annotations)
p2_dataset = Dataset.from_pandas(p2_df)

In [65]:
p2_dataset = p2_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [66]:
# keep just sentences with the same length
sentences_to_evaluate_p2 = []
sentences_to_evaluate_true = []

for i in range(len(p2_dataset)):
    if len(p2_dataset['labels'][i]) == len(true_ann_dataset['labels'][i]):
        sentences_to_evaluate_p2.append(p2_dataset['labels'][i])
        sentences_to_evaluate_true.append(true_ann_dataset['labels'][i])

print(len(sentences_to_evaluate_p2)/len(p2_dataset))

0.72


In [67]:
evaluator = BioEval()

In [68]:
pred_labels, true_labels = get_labels((sentences_to_evaluate_p2, sentences_to_evaluate_true))

In [71]:
evaluator.evaluate_annotations(true_labels, pred_labels, do_lower=True)

In [72]:
evaluator.performance

{'overall': {'acc': 0.6897,
  'strict': {'precision': 0.628158844765343,
   'recall': 0.5958904109589042,
   'f_score': 0.6115992970123024},
  'relax': {'precision': 0.7075812274368231,
   'recall': 0.6712328767123288,
   'f_score': 0.6889279437609842}},
 'category': {'strict': {'drug': {'precision': 0.15384615384615385,
    'recall': 0.11764705882352941,
    'f_score': 0.13333333333333333},
   'condition': {'precision': 0.7056277056277056,
    'recall': 0.6965811965811965,
    'f_score': 0.7010752688172043},
   'measurement': {'precision': 0.25, 'recall': 0.25, 'f_score': 0.25},
   'temporal': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0},
   'procedure': {'precision': 0.42857142857142855,
    'recall': 0.5454545454545454,
    'f_score': 0.4799999999999999},
   'value': {'precision': 0.5,
    'recall': 0.14285714285714285,
    'f_score': 0.22222222222222224},
   'observation': {'precision': 0.0, 'recall': 0.0, 'f_score': 0.0},
   'person': {'precision': 0.0, 'recall': 0.0, 'f_scor

In [73]:
evaluator.save_evaluation('eval_p2.json')

In [74]:
evaluator.get_counts()

{'expect': {'overall': 292,
  'drug': 17,
  'condition': 234,
  'measurement': 8,
  'value': 7,
  'procedure': 11,
  'temporal': 11,
  'observation': 3,
  'person': 1},
 'prediction': {'strict': {'overall': {'total': 277,
    'true': 174,
    'false': 103},
   'drug': {'total': 13, 'true': 2, 'false': 11},
   'condition': {'total': 231, 'true': 163, 'false': 68},
   'measurement': {'total': 8, 'true': 2, 'false': 6},
   'temporal': {'total': 3, 'true': 0, 'false': 3},
   'procedure': {'total': 14, 'true': 6, 'false': 8},
   'value': {'total': 2, 'true': 1, 'false': 1},
   'observation': {'total': 2, 'true': 0, 'false': 2},
   'person': {'total': 4, 'true': 0, 'false': 4}},
  'relax': {'overall': {'total': 277, 'true': 196, 'false': 81},
   'drug': {'total': 13, 'true': 4, 'false': 9},
   'condition': {'total': 231, 'true': 180, 'false': 51},
   'measurement': {'total': 8, 'true': 2, 'false': 6},
   'temporal': {'total': 3, 'true': 2, 'false': 1},
   'procedure': {'total': 14, 'true': 7